# Wavelenth calibration

This notebook assumes you've completed the Spectroscopic Trace process and have a trace handy.

In [ ]:
from PIL import Image
import numpy as np
import pylab as pl
pl.rcParams['image.origin'] = 'lower'
pl.matplotlib.style.use('dark_background')

We have three calibration lamp spectra: Mercury, Krypton, and Neon.

In [ ]:
hg_filename = "hg_lamp_1-sixteenth_s.bmp"
kr_filename = "kr_lamp_p6.bmp"
ne_filename = "ne_lamp_1s.bmp"

In [ ]:
hg_image = np.array(Image.open(hg_filename))
kr_image = np.array(Image.open(kr_filename))
ne_image = np.array(Image.open(ne_filename))

In [ ]:
pl.imshow(hg_image)

In [ ]:
pl.imshow(kr_image)

In [ ]:
pl.imshow(ne_image)

We can note by zooming in that there appear to be seven independent spectra:

In [ ]:
pl.imshow(hg_image[380:540,140:180])

We re-create our trace model from the [Spectroscopic Trace Tutorial](Spectroscopic%20Trace%20Tutorial.ipynb) using the fitted models.

(we could have used the empirical trace directly, which might result in slightly improved noise characteristics, but for simplicity - and to make the two notebooks independently usable - we use the fitted polynomial & Gaussian models here)

In [ ]:
from astropy.modeling.polynomial import Polynomial1D
from astropy.modeling.models import Gaussian1D
trace_model = Polynomial1D(degree=2, c0=453.99089409, c1=-0.01931577, c2=0.00000731,)
trace_profile_model = Gaussian1D(amplitude=123.81906886, mean=0.17890498, stddev=5.10870321)
xaxis = np.arange(hg_image.shape[1])
trace_center = trace_model(xaxis)
npixels_to_cut=15
yaxis = np.arange(-npixels_to_cut, npixels_to_cut)
model_trace_profile = trace_profile_model(yaxis)

Then, we use the trace to extract spectra from each of the three calibration lamp spectra.

We verify that the traces look acceptable first, though:

In [ ]:
pl.figure(figsize=(16,8))
ax1 = pl.subplot(3,1,1)
ax1.imshow(hg_image[430:470,:], extent=[0,1600,430,470])
ax1.plot(xaxis, trace_center)
ax1.set_aspect(10)
ax2 = pl.subplot(3,1,2)
ax2.imshow(kr_image[430:470,:], extent=[0,1600,430,470])
ax2.plot(xaxis, trace_center)
ax2.set_aspect(10)
ax3 = pl.subplot(3,1,3)
ax3.imshow(ne_image[430:470,:], extent=[0,1600,430,470])
ax3.plot(xaxis, trace_center)
ax3.set_aspect(10)

Strangely, the trace looks like it's going through a minimum between the individual spectra, but there's enough signal that we can extract a calibration lamp spectrum, and the trace is perpendicular enough to the dispersion axis that we can mostly trust it.  Nonetheless, you might question whether these observations were carried out correctly, or if the spectrograph was shaken severely between star & cal lamp observations.

In [ ]:
hg_spectrum = np.array([np.average(hg_image[int(yval)-npixels_to_cut:int(yval)+npixels_to_cut, ii],
                                weights=model_trace_profile)
                     for yval, ii in zip(trace_center, xaxis)])

In [ ]:
ne_spectrum = np.array([np.average(ne_image[int(yval)-npixels_to_cut:int(yval)+npixels_to_cut, ii],
                                weights=model_trace_profile)
                     for yval, ii in zip(trace_center, xaxis)])

In [ ]:
kr_spectrum = np.array([np.average(kr_image[int(yval)-npixels_to_cut:int(yval)+npixels_to_cut, ii],
                                weights=model_trace_profile)
                     for yval, ii in zip(trace_center, xaxis)])

In [ ]:
_=pl.plot(xaxis, hg_spectrum, label='Mercury')
_=pl.plot(xaxis, ne_spectrum, label='Neon')
_=pl.plot(xaxis, kr_spectrum, label='Krypton')
_=pl.legend(loc='best')

We now have spectra of a mercury lamp, a neon lamp, and a krypton lamp.

If we have no prior knowledge, we have to do some guess-and-check, but wikipedia's a decent resource to point us at the right line lists:

 * https://en.wikipedia.org/wiki/Mercury-vapor_lamp#Emission_line_spectrum
 * https://en.wikipedia.org/wiki/Gas-discharge_lamp#Color

In [ ]:
from IPython.display import Image

In [ ]:
Image("https://upload.wikimedia.org/wikipedia/commons/2/29/Mercury_Spectra.jpg")

In [ ]:
Image("https://upload.wikimedia.org/wikipedia/commons/9/99/Neon_spectra.jpg")

In [ ]:
Image("https://upload.wikimedia.org/wikipedia/commons/a/a6/Krypton_Spectrum.jpg")

In [ ]:
from astropy import units as u

Start with mercury...

In [ ]:
pl.plot(xaxis, hg_spectrum)

In [ ]:
# Guessing these is hard and takes experience
# I figured this out by noting that an A-star we observed, Deneb, had an absorption line around pixel ~750
# That absorption line pretty much has to be H-alpha or H-beta
guessed_wavelengths = [546.1, 435.8, 404.7]
guessed_xvals = [165, 1230, 1550]

We can do a lot better at determining the X-values by taking the intensity-weighted coordinate (moment 1):

In [ ]:
npixels = 15
improved_xval_guesses = [np.average(xaxis[g-npixels:g+npixels],
                                    weights=hg_spectrum[g-npixels:g+npixels] - np.median(hg_spectrum))
                         for g in guessed_xvals]
improved_xval_guesses

In [ ]:
pl.plot(xaxis, hg_spectrum)
pl.plot(guessed_xvals, [100]*3, 'x')
pl.plot(improved_xval_guesses, [100]*3, '+')

We only have three data points, but that is enough to fit a linear model and still have a free point to check that we got it close to right:

In [ ]:
from astropy.modeling.fitting import LinearLSQFitter
linfitter = LinearLSQFitter()

We use a `Linear1D` model because we will want to use its inverse later

In [ ]:
from astropy.modeling.models import Linear1D
wlmodel = Linear1D()
linfit_wlmodel = linfitter(model=wlmodel, x=improved_xval_guesses, y=guessed_wavelengths)
wavelengths = linfit_wlmodel(xaxis) * u.nm
linfit_wlmodel

In [ ]:
pl.plot(wavelengths, hg_spectrum)
pl.plot(guessed_wavelengths, [100]*3, 'x')

In [ ]:
pl.plot(improved_xval_guesses, guessed_wavelengths, 'o')
pl.plot(xaxis, wavelengths, '-')

Indeed, a linear model fit excellently!

We can look up neon and krypton line lists to see how well we did and maybe try to improve our fit.

NIST, the National Institute of Standards and Technology, maintains [atomic line lists](https://physics.nist.gov/PhysRefData/ASD/lines_form.html).  However, we have no direct way to know which transitions from these atoms to use!  There are a few heuristics we can apply, though.

In [ ]:
# astroquery provides an interface to the NIST atomic line database
from astroquery.nist import Nist
# we adopt the minimum/maximum wavelength from our linear fit
minwave = wavelengths.min()
maxwave = wavelengths.max()
# then we search for atomic lines
# We are only interested in neutral lines, assuming the lamps are not hot enough to ionize the atoms
mercury_lines = Nist.query(minwav=minwave, maxwav=maxwave, linename='Hg I')
krypton_lines = Nist.query(minwav=minwave, maxwav=maxwave, linename='Kr I')
neon_lines = Nist.query(minwav=minwave, maxwav=maxwave, linename='Ne I')

We can first look at what NIST reports for the mercury spectrum we already fit:

(note that the wavelengths are *air* wavelengths, not vacuum)

In [ ]:
pl.plot(wavelengths, hg_spectrum)
pl.vlines(mercury_lines['Observed'], 0, 250, 'w', alpha=0.25)

The database contains *many* more lines than we saw.

The "Rel. Int" (Relative Intensity) provides "a qualitative description of what the emission spectrum of a particular element in a particular (low-density) source looks like." 

We can select that column, if we clean it up a little (there are some "blank" entries)

In [ ]:
hg_keep = (~mercury_lines['Rel.'].mask) & (mercury_lines['Rel.'] != "*")
hg_wl_tbl = mercury_lines['Observed'][hg_keep]
hg_rel_tbl = np.array([float(x) for x in mercury_lines['Rel.'][hg_keep]])

In [ ]:
pl.plot(wavelengths, hg_spectrum)
pl.plot(hg_wl_tbl, hg_rel_tbl / hg_rel_tbl.max() * hg_spectrum.max(), 'x')

Not bad!  See that only the three "brightest" lines show up, plus maybe a bit of a third one around 408 nm.

Let's try the same for the other lines (note that the hackery changes - this is because the NIST tables are not formatted to be machine-readable!)

In [ ]:
ne_keep = np.array(['*' not in x for x in neon_lines['Rel.']])
ne_wl_tbl = neon_lines['Observed'][ne_keep]
ne_rel_tbl = np.array([float(x) for x in neon_lines['Rel.'][ne_keep]])

In [ ]:
kr_keep = np.array([('*' not in x) and ('h' not in x) and ('-' not in x) for x in krypton_lines['Rel.']])
kr_wl_tbl = krypton_lines['Observed'][kr_keep]
kr_rel_tbl = np.array([float(x) for x in krypton_lines['Rel.'][kr_keep]])

In [ ]:
pl.plot(wavelengths, ne_spectrum)
pl.plot(ne_wl_tbl, ne_rel_tbl / ne_rel_tbl.max() * ne_spectrum.max(), 'x')

In [ ]:
pl.plot(wavelengths, kr_spectrum)
pl.plot(kr_wl_tbl, kr_rel_tbl / kr_rel_tbl.max() * kr_spectrum.max(), 'x')

The Krypton spectrum will be much easier to deal with: we just select those lines with intensities > 70 on the scale above:

In [ ]:
kr_rel_intens =  kr_rel_tbl / kr_rel_tbl.max() * kr_spectrum.max()
kr_keep_final = kr_rel_intens > 70
kr_wl_final = kr_wl_tbl[kr_keep_final]

We then take these krypton wavelengths and convert them back to pixel space:

In [ ]:
kr_pixel_vals = linfit_wlmodel.inverse(kr_wl_final)

In [ ]:
pl.plot(xaxis, kr_spectrum)
pl.plot(kr_pixel_vals, kr_rel_intens[kr_keep_final], 'x')

As with mercury above, we select a sub-region and measure the intensity-weighted peak...

But here we have to be careful!  These peaks are very close!  Can we still use a +/-15 pixel range?

In [ ]:
pl.plot(xaxis, kr_spectrum)
pl.plot(kr_pixel_vals, kr_rel_intens[kr_keep_final], 'x')
for xx, yy in zip(kr_pixel_vals, kr_rel_intens[kr_keep_final]):
    pl.plot([xx-15,xx+15], [yy,yy], )
pl.xlim(1200,1350)

Yes, 15 pixels looks OK still.

In [ ]:
# note that we have to force the pixel-center guesses to be integers for slicing:
npixels = 15
improved_xval_guesses_kr = [np.average(xaxis[g-npixels:g+npixels],
                                    weights=kr_spectrum[g-npixels:g+npixels] - np.median(kr_spectrum))
                         for g in map(int, kr_pixel_vals)]
improved_xval_guesses_kr

Now we can plot these best-fit pixel numbers against the "true" wavelengths and overplot our previous best-fit model from mercury:

In [ ]:
pl.plot(improved_xval_guesses_kr, kr_wl_final, 'x', label='Krypton')
pl.plot(improved_xval_guesses, guessed_wavelengths, '+', label='Mercury')
pl.plot(xaxis, wavelengths, zorder=-5)
pl.legend(loc='best')

We can now jointly fit the Kr + Hg spectra with a new model:

In [ ]:
xvals_hg_plus_kr = list(improved_xval_guesses) + list(improved_xval_guesses_kr)
waves_hg_plus_kr = list(guessed_wavelengths) + list(kr_wl_final)
linfit_wlmodel_hgkr = linfitter(model=wlmodel, x=xvals_hg_plus_kr, y=waves_hg_plus_kr)
linfit_wlmodel_hgkr

In [ ]:
pl.plot(improved_xval_guesses_kr, kr_wl_final, 'x', label='Krypton')
pl.plot(improved_xval_guesses, guessed_wavelengths, '+', label='Mercury')
pl.plot(xaxis, wavelengths, zorder=-5)
pl.plot(xaxis, linfit_wlmodel_hgkr(xaxis), zorder=-5)
pl.legend(loc='best')

These are nearly indistiguishable, but let's look at the residuals for each:

In [ ]:
residuals_hgonly = np.array(waves_hg_plus_kr) - linfit_wlmodel(xvals_hg_plus_kr)
residuals_hgkr = np.array(waves_hg_plus_kr) - linfit_wlmodel_hgkr(xvals_hg_plus_kr)
pl.hist(residuals_hgonly, label='Hg-only fit', histtype='stepfilled')
pl.hist(residuals_hgkr, label='Hg-only fit', histtype='step')

The differences are pretty marginal; the Hg-only fit was really good.

Is there any shape to the residuals?

In [ ]:
pl.plot(xvals_hg_plus_kr, residuals_hgonly, 'x')

In [ ]:
pl.plot(xvals_hg_plus_kr, residuals_hgkr, 'x')

In brief, no, there's no sign of structure.

We can also estimate a chi-squared-like statistic (though note that this is *not* the $\chi^2$ value we would use to assess goodness of fit because we have *not* estimated the uncertainty on each wavelength point!)

In [ ]:
resid2_hgonly = (residuals_hgonly**2).sum()
resid2_hgkr = (residuals_hgkr**2).sum()
resid2_hgonly, resid2_hgkr

These differences are also very small, again suggesting the Hg-only fit was really good.

Why do we bother going through all these checks if the Hg-only fit was just fine?  

Besides simply not knowing, it is common for spectrographs to have slightly non-linear dispersion when measured over a wide wavelength range, so we want to be sure we haven't missed any such curvature.

We also want to measure the uncertainty on our wavelength calibration.

Now let's go back and measure the Neon spectrum

In [ ]:
ne_rel_intens = ne_rel_tbl / ne_rel_tbl.max() * ne_spectrum.max()
pl.plot(wavelengths, ne_spectrum)
pl.plot(ne_wl_tbl, ne_rel_intens, 'x')

We can try keeping only those with intensiy >100, though we will miss some lines

In [ ]:
ne_keep_final = ne_rel_intens > 100
pl.plot(wavelengths, ne_spectrum)
pl.plot(ne_wl_tbl[ne_keep_final], ne_rel_intens[ne_keep_final], 'x')

In [ ]:
ne_wl_final = ne_wl_tbl[ne_keep_final]
ne_pixel_vals = linfit_wlmodel.inverse(ne_wl_final)

In [ ]:
pl.plot(xaxis, ne_spectrum)
pl.plot(ne_pixel_vals, ne_rel_intens[ne_keep_final], 'x')
for xx, yy in zip(ne_pixel_vals, ne_rel_intens[ne_keep_final]):
    pl.plot([xx-15,xx+15], [yy,yy], )
pl.xlim(600,850)

For Neon, our automated approach isn't very good (the third point from the left is not on a peak), and we'll want to use a narrower pixel range.

In [ ]:
# note that we have to force the pixel-center guesses to be integers for slicing:
npixels = 10
improved_xval_guesses_ne = [np.average(xaxis[g-npixels:g+npixels],
                                    weights=ne_spectrum[g-npixels:g+npixels] - np.median(ne_spectrum))
                         for g in map(int, ne_pixel_vals)]
improved_xval_guesses_ne

In [ ]:
pl.plot(improved_xval_guesses_kr, kr_wl_final, 'x', label='Krypton')
pl.plot(improved_xval_guesses_ne, ne_wl_final, '^', label='Neon')
pl.plot(improved_xval_guesses, guessed_wavelengths, '+', label='Mercury')
pl.plot(xaxis, wavelengths, zorder=-5)
pl.plot(xaxis, linfit_wlmodel_hgkr(xaxis), zorder=-5)
pl.legend(loc='best')

In [ ]:
xvals_hg_plus_kr_plus_ne = list(improved_xval_guesses) + list(improved_xval_guesses_kr) + list(improved_xval_guesses_ne)
waves_hg_plus_kr_plus_ne = list(guessed_wavelengths) + list(kr_wl_final) + list(ne_wl_final)
linfit_wlmodel_hgkrne = linfitter(model=wlmodel, x=xvals_hg_plus_kr_plus_ne, y=waves_hg_plus_kr_plus_ne)
linfit_wlmodel_hgkrne

In [ ]:
residuals_hgonly = np.array(waves_hg_plus_kr_plus_ne) - linfit_wlmodel(xvals_hg_plus_kr_plus_ne)
residuals_hgkr = np.array(waves_hg_plus_kr_plus_ne) - linfit_wlmodel_hgkr(xvals_hg_plus_kr_plus_ne)
residuals_hgkrne = np.array(waves_hg_plus_kr_plus_ne) - linfit_wlmodel_hgkrne(xvals_hg_plus_kr_plus_ne)

In [ ]:
resid2_hgonly = (residuals_hgonly**2).sum()
resid2_hgkr = (residuals_hgkr**2).sum()
resid2_hgkrne = (residuals_hgkrne**2).sum()
resid2_hgonly, resid2_hgkr, resid2_hgkrne

In [ ]:
pl.plot(xvals_hg_plus_kr_plus_ne, residuals_hgkrne, 'x')

Finally, with neon included, there is a hint of some structure

In [ ]:
_=pl.hist(residuals_hgkrne)
_=pl.title(f"Stddev = {residuals_hgkrne.std()}")

This histogram is perhaps a little skewed.

There may be some curvature to this spectrum, but the evidence is not terribly compelling, particularly given the uncertain nature of the Neon line fits.

## What is our uncertainty?

To evaluate our uncertainty, let's first look at the uncertainty in the pixel value inferred from looking at a single line in the mercury spectrum.

In [ ]:
pl.plot(xaxis, hg_spectrum)
pl.xlim(100,200)

We can get a *coarse* empirical estimate of the per-pixel uncertainty by taking the standard deviation of a "blank" part of the spectrum:

In [ ]:
pl.plot(xaxis[200:300], hg_spectrum[200:300])
noise_estimate = hg_spectrum[200:300].std()
_=pl.title(f"stddev = {noise_estimate:0.2f}")

The moment-1 estimate of the peak location is, for a spectrum $f(x)$ and pixel location $x$ 
$$ m_1 = \frac{\Sigma x f(x)}{\Sigma f(x)} $$

which, through propagation of error, gives us variance of moment 1:

$$\sigma_{m_1}^2 =  \left(\frac{\Sigma \left[ (x-m_1)^2   \sigma_{f(x)}^2 \right]}{\left(\Sigma f(x)\right)^2}  +
   \frac{\sigma_{\Sigma f(x)}^2 \Sigma \left[ (x-m_1)^2 f(x)^2 \right]}{\left(\Sigma f(x)\right)^4} \right)$$
   
   where 
$   \sigma_{\Sigma f(x)}^2 = \Sigma \sigma_{f(x)}^2 = N \sigma_{f(x)}^2 $

In [ ]:
cutout = hg_spectrum[100:200] - np.median(hg_spectrum)
xcutout = xaxis[100:200]
m1 = (xcutout * cutout).sum() / cutout.sum()
m1

In [ ]:
sigma_m1_left = ((xcutout-m1)**2 * noise_estimate**2).sum() / cutout.sum()**2
sigma_m1_right = (xcutout.size * noise_estimate**2) * ((xcutout-m1)**2*cutout**2).sum() / cutout.sum()**4
sigma_m1 = sigma_m1_left + sigma_m1_right
sigma_m1

In [ ]:
print(f"Moment analysis yields m1 = {m1:0.3f} +/- {sigma_m1:0.3f})")

The uncertainty is about 1/1000th of a pixel for a bright line.  It may be somewhat larger for fainter lines, but it is still much smaller than our observed scatter.  We can prove this by looking at a faint neon line:

In [ ]:
cutoutne = ne_spectrum[640:660] - np.median(ne_spectrum)
xcutoutne = xaxis[640:660]
m1ne = (xcutoutne * cutoutne).sum() / cutoutne.sum()

In [ ]:
ne_noise_estimate = ne_spectrum[1400:1600].std()
ne_noise_estimate

In [ ]:
pl.plot(xcutoutne, cutoutne)
pl.plot(m1ne, cutoutne.max(), 'x')

In [ ]:
sigma_m1_left = ((xcutoutne-m1ne)**2 * ne_noise_estimate**2).sum() / cutoutne.sum()**2
sigma_m1_right = (xcutoutne.size * ne_noise_estimate**2) * ((xcutoutne-m1ne)**2*cutoutne**2).sum() / cutoutne.sum()**4
sigma_m1_ne = sigma_m1_left + sigma_m1_right
sigma_m1_ne

This error is ~40 times larger, but still much less than a pixel's width.

Our best fit gives that each pixel is about 0.1 nm, so our uncertainty is 0.0001 nm - 0.004 nm.  The scatter in the residuals is 0.23 nm, so it is not explained by our formal fit's statistical uncertainty. 

## What does the residual represent?

The residual to our fit gives us a sense of the *systematic* uncertainty from a combination of line blending and imperfect fits.  We know, from the NIST line lists, that there are in some cases multiple lines that we could not easily distinguish in the spectrum: these add a small bias to our inferred pixel locations.

There may also be subtle effects from the spectrograph itself that cause small systematic variations in the wavelength solution.

## How could we do better?

We could attempt to *deblend* the individual spectral features by jointly fitting multiple-Gaussian models.

While this procedure is commonly used, it is fraught with error and extremely difficult to automate in general.